# Homework Assignment: Chaining API Requests

**Course:** Introduction to Data Science
**Topic:** Working with Web APIs

### The Goal

In class, we learned how to send an API request with parameters and authentication. In this assignment, we will take it a step further: **chaining API requests**.

**Your Mission:** Write a script that takes a country's name (e.g., "France"), automatically finds its capital city (e.g., "Paris"), and then fetches the *current* weather for that capital city.

To do this, we will use two different APIs:
1.  **REST Countries**: To find the capital of a country (no key required).
2.  **OpenWeatherMap**: To get the weather in a city (requires the API key you generated in class).

This is a graded assignment. You must fill in the missing code in the `Your Turn` cells and submit the notebook with all outputs visible and correct.

### Part 1: Setup and Configuration

Let's install and import all the libraries we'll need.

In [2]:
# 1. Install the required libraries
%pip install requests

In [4]:
import requests

# --- (Your Turn: Task 1) --- 
# 2. Set your OpenWeatherMap API key variable
# (This is the same key you used in the class exercise)
OWM_API_KEY = "75bba0613fb83b6d30267623cc6cb43f"

# 3. Set the base URLs for both API services
COUNTRIES_API_URL = "https://restcountries.com/v3.1/name/"
WEATHER_API_URL = "https://api.openweathermap.org/data/2.5/weather"

print("Setup complete.")

Setup complete.


### Part 2: Task 2.1 - Find the Capital City

**Your Task:** Write a function `get_capital(country_name)`.
This function should take a country name, make an API call to `REST Countries`, and return the name of its capital city.

**Hints:**
* The full URL will look something like: `https://restcountries.com/v3.1/name/Germany`.
* The response is a **list**. You will need to access the first element `[0]`.
* The capital city field is at `data[0]['capital'][0]`.
* Don't forget error handling! Use `try...except` and check the response `status_code`.

In [5]:
# --- (Your Turn: Task 2) --- 

def get_capital(country_name):
    """
    Gets a country name and returns its capital city using REST Countries API.
    """
    try:
        # 1. Make the .get() request to the correct URL
        # (Combine COUNTRIES_API_URL with country_name)
        response = requests.get(f"{COUNTRIES_API_URL}{country_name}")
        
        # 2. Check if the request was successful (status_code == 200)
        if response.status_code == 200:
            data = response.json()
            
            # 3. Extract the capital name (Hint: data[0]['capital'][0])
            capital_name = data[0]['capital'][0]
            return capital_name
        else:
            # 4. If the request failed, return an error message
            return f"Error: Country '{country_name}' not found."
            
    except Exception as e:
        return f"Error: An exception occurred - {e}"

# --- Test your function --- 
print(f"Capital of Germany: {get_capital('Germany')}")
print(f"Capital of Japan: {get_capital('Japan')}")
print(f"Capital of FakeCountry: {get_capital('FakeCountry')}")

Capital of Germany: Berlin
Capital of Japan: Tokyo
Capital of FakeCountry: Error: Country 'FakeCountry' not found.


### Part 3: Task 2.2 - Get the Weather

**Your Task:** Write a function `get_weather(city_name)`.
This is a review of the class exercise. The function takes a city name and uses the OpenWeatherMap API to return a short weather description.

**Hints:**
* You will need to build a `params` dictionary containing `q`, `appid`, and `units` (set this to `metric`).
* Temperature is at `data['main']['temp']`.
* Weather description is at `data['weather'][0]['description']`.

In [6]:
# --- (Your Turn: Task 3) --- 

def get_weather(city_name):
    """
    Gets a city name and returns a string with its current weather.
    Uses the global OWM_API_KEY.
    """
    
    # 1. Define the parameters dictionary (params) 
    # It must include: q, appid, units
    params = {
        'q': city_name,
        'appid': OWM_API_KEY,
        'units': 'metric'
    }
    
    try:
        # 2. Make the .get() request to WEATHER_API_URL with the params
        response = requests.get(WEATHER_API_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            
            # 3. Extract the temperature and description
            temp = data['main']['temp']
            desc = data['weather'][0]['description']
            
            return f"The weather in {city_name} is {temp}°C with {desc}."
        else:
            return f"Error: Weather data for '{city_name}' not found."
            
    except Exception as e:
        return f"Error: An exception occurred - {e}"

# --- Test your function --- 
print(get_weather("London"))
print(get_weather("Tel Aviv"))

The weather in London is 2.48°C with scattered clouds.
The weather in Tel Aviv is 26.27°C with clear sky.


### Part 4: Task 2.3 - Chain the Requests

**Your Task:** This is the main event! Write a final function `get_capital_weather(country_name)`.

This function must:
1.  Call your `get_capital(country_name)` function to get the capital city's name.
2.  Take the returned capital city name and pass it as an argument to your `get_weather()` function.
3.  Return the final result (the weather string).

**Important:** Remember to handle cases where the first call (`get_capital`) returns an error. If it does, the second function should not even run.

In [9]:
# --- (Your Turn: Task 4) --- 

def get_capital_weather(country_name):
    """
    Chains the two API calls together.
    """
    # 1. Get the capital city
    capital = get_capital(country_name)
    
    # 2. Check if the first call returned an error.
    # An easy way is to check if the string 'Error:' is in the result.
    if "Error:" in capital:
        # 3. If it's an error, just return that error message
        return capital
    
    # 4. If it's not an error, 'capital' holds a valid city name.
    # Now, get the weather for that capital.
    weather_report = get_weather(capital)
    
    # 5. Return the final report
    return weather_report

# --- Test your final function! --- 
print("--- Final Test --- ")
print(get_capital_weather("Italy"))
print(get_capital_weather("Canada"))
print(get_capital_weather("Australia"))
print(get_capital_weather("A non-existent country"))

--- Final Test --- 
The weather in Rome is 3.82°C with clear sky.
The weather in Ottawa is -1.76°C with overcast clouds.
The weather in Canberra is 18.03°C with clear sky.
Error: Country 'A non-existent country' not found.
